# Introduction
* [Episode 1 - Transceivers and Filters - Part 1](https://www.youtube.com/watch?v=r_p7AHsSOdw&list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY&index=1&pp=iAQB)
* [Episode 1 - Transceivers and Filters - Part 2](https://www.youtube.com/watch?v=He0-X6FCLMo&list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY&index=2&pp=iAQB)

## Pre-select Filter
### Parallel LC Bandpass Filter

In [3]:
import math
f_0 = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
Rin = 50 # 50 Ohm, input impedance (antenna impedance)
Rout = 50 # 50 Ohm, output impedance (input impedance of the amplifier)

R_L = Rin * Rout / (Rin + Rout) # l
Q = f_0 / B # Q factor
print(f"Q = {Q}")
X_0 = R_L / Q
ω = 2 * math.pi * f_0

L = X_0 / ω # inductance
print(f"Calculated L = {L} H")

# Select the actual part for the inductor:
L_act = 10e-9 # 10 nH, WE 744917110
Q_Lact = 39 # Q factor of the actual inductor
R_Lact = ω * L_act / Q_Lact # resistance of the actual inductor
print(f"R_Lact = {R_Lact} Ohm")

# Calculate the capacitance
C = 1 / (L_act * ω**2) # capacitance 
print(f"C = {C} F")

Q = 4.9
Calculated L = 8.28586750790792e-09 H
R_Lact = 0.15788516925733317 Ohm
C = 2.6374735433761393e-10 F


The 264pF capacitor for the pre-select filter will be constituated of a 47pF and a 220pF capacitor in parallel.  A 180pF+82pF is possible too, but these values are not available at Wurth Elektronik.

* L = 10nH, WE 744917110
* C1 = 47pF, WE 885012006055
* C2 = 220pF, WE 885012006059

### Top-C Coupled resonator band pass filter
This topology is proposed by the lecturer in the video series, but it's not used.

[Filter Design Part 11. How to Design Bandpass Filter BPF with Top-Coupled Capacitive Resonator.](https://www.youtube.com/watch?v=djknXHFLqsA) gives some details about when to use this topology.  
* low side roll/off 18dB/octave
* high side roll/off 6dB/octave.  

This makes this filter more suitable for low-side injection in which the frequency of the local oscillator is lower than the frequency of the signal.  The unwanted image frequency is lower than the wanted frequency, so it gets more suppression.

In [ ]:
import math
# Calculations from [Filter Design Part 11. How to Design Bandpass Filter BPF with Top-Coupled Capacitive Resonator.](https://www.youtube.com/watch?v=djknXHFLqsA)
# The same calculations can also be found on [RF Circuit Design, Second Edition -- Christopher Bowick, Chapter 2, Coupling of resonant circuits]

f_0 = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
Rs = Rl = 50 # 50 Ohm, source and load impedance

Q_t = f_0 / B # Q factor of the total filter
Q_r = Q_t * math.sqrt(2) # Q factor of a single resonator
X_L1 = Rs / Q_r # inductive reactance
X_C1 = X_L1 # capacitive reactance
L_1 = X_L1 / (2 * math.pi * f_0) # inductance of the resonator
print(f"L_1 = {L_1} H")
C_1 = 1 / (2 * math.pi * f_0 * X_C1) # capacitance of the resonator
print(f"C_1 = {C_1} F")
C_c = C_1 / Q_r # coupling capacitance
print(f"C_c = {C_c} F")

L_1 = 1.1717986205709938e-08 H
C_1 = 2.2507907903927656e-10 F
C_c = 3.248060063099905e-11 F


When simulating this filter in LTSpice, the resulting frequency response shows a center frequency considerably lower than 98MHz.  I have no idea why.

Playing around a bit with the component values in LTSpice delivered a filter with the following parameters:
* Center frequency: 98MHz
* Bandwidth : 20MHz
* L||C = 15nH||150pF
* Coupling capacitor: 33pF

### Conventional, shunt first band pass filter
* Butterworth, 3rd order
* Chebyshev, 3rd order

## Simulation

<figure>
    <img src='./simulation/pre-selector-filter_schematic.png' alt='missing' />
    <figcaption>Schematic of the three filter topologies: OUT1=parallel LC, OUT3=top C-coupled resonator, OUT4=Conventional shunt first BPF</figcaption>
</figure>

<figure>
    <img src='./simulation/pre-selector-filter.png' alt='missing' />
    <figcaption>Simulation results of the three filter topologies</figcaption>
</figure>

### Simulation results of the parallel LC bandpass filter
* Center frequency: 95.5MHz
* Insertion loss: 1.4dB
* Return loss : > 27dB
* Lower 3dB frequency: 83.4MHz
* Upper 3dB frequency: 108.9MHz